In [1]:

# Initial Setup:
import pandas as pd
import numpy as np
import re
from datetime import datetime

url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
df = pd.read_excel(url)

In [2]:
# Preview of raw DataFrame:
print("head:")
print(df.head())

print("\ninfo:")
df.info()

print("\ndescribe:")
print(df.describe())

print("\ncolumns:")
print(df.columns.tolist())

head:
          Date    Year        Type    Country              State  \
0  15 Mar 2024  2024.0  Unprovoked  AUSTRALIA         Queensland   
1  04 Mar 2024  2024.0  Unprovoked        USA             Hawaii   
2  02 Mar-2024  2024.0  Unprovoked        USA             Hawaii   
3  25 Feb-2024  2024.0  Unprovoked  AUSTRALIA  Western Australia   
4  14 Feb-2024  2024.0  Unprovoked      INDIA        Maharashtra   

                           Location  Activity                 Name Sex  Age  \
0                     Bargara Beach  Swimming       Brooklyn Sauer   F   13   
1                Old Man's, Waikiki   Surfing        Matthew White   M  NaN   
2                    Rainbows, Oahu  Swimming                  NaN   F   11   
3        Sandlnd Island, Jurian Bay       NaN               female   F   46   
4  Vaitarna River, Palghar District   Fishing  Vicky Suresh Govari   M   32   

   ...        Species                      Source  pdf href formula href  \
0  ...     Tiger shark      Yahoo 

In [3]:
# DataFrame cleaning preparation:

# 1. Dropping unneeded columns and duplicates:
columns_to_drop = ["Source", "Location", "Injury", "Name", "pdf", "href formula",
                   "href", "Case Number", "Case Number.1", "original order",
                   "Unnamed: 21", "Unnamed: 22", "Species "]

df = df.drop(columns=columns_to_drop).drop_duplicates().reset_index(drop=True)

# 2. Filter rows with Year > 1800 and remove unneeded 'Types':
df = df[df["Year"] > 1800]

# 3. Types of shark attacks to exclude:
undesired_types = ["Questionable", "Boat", "Provoked", "Provoked ", "?",
                   "Unverified", "Under investigation", "Unconfirmed"]

df = df[~df["Type"].isin(undesired_types)]

# 4. Renaming and reformatting columns:
df.columns = [col.strip().replace(" ", "_").replace(".", "").lower() for col in df.columns]
df.rename(columns={'unnamed:_11': 'fatal'}, inplace=True)

# 5. Creating a copy of the original DataFrame for further manipulation
df_copy = df.copy()

# Check:
print(df.columns.tolist())

['date', 'year', 'type', 'country', 'state', 'activity', 'sex', 'age', 'fatal', 'time']


In [4]:
# Cleaning 'fatal' column:
value_map = {'n': 'n', 'y': 'y'}

df_copy['fatal'] = df_copy['fatal'].str.strip().str.lower().map(value_map)

# Calculate mode:
fatal_mode = df_copy['fatal'].mode()[0]

# Replace NaNs with 'n':
df_copy['fatal'] = df_copy['fatal'].fillna(fatal_mode)

# Check:
df_copy['fatal'].value_counts(dropna=False)

fatal
n    4740
y    1344
Name: count, dtype: int64

In [5]:
# Cleaning 'sex' column:
df_copy['sex'] = df_copy['sex'].str.strip().str.lower()

# Replace wrong values with 'unknown':
invalid_entries = ['lli', 'm x 2', 'n', '.']
for entry in invalid_entries:
    df_copy['sex'] = df_copy['sex'].replace(entry, 'unknown')

# Replace missing values with 'unknown':
df_copy['sex'].fillna('unknown')

# Check:
df_copy['sex'].value_counts(dropna=False)

sex
m          4856
f           729
NaN         495
unknown       4
Name: count, dtype: int64

In [6]:
#Cleaning 'type' column:

# Remove 'invalid' values:
df_copy = df_copy[df_copy['type'] != 'invalid']

# Replace 'NaN' with 'unknown':
df_copy['type'].fillna('unknown')

# Check:
df_copy['type'].value_counts(dropna=False)

type
Unprovoked      4937
Invalid          546
Watercraft       349
Sea Disaster     234
NaN               16
 Provoked          2
Name: count, dtype: int64

In [7]:
# unique_values = df['time'].unique()
# print(unique_values)

In [8]:
# 1. Convert 'time' column to strings (to avoid issues with NaN when splitting)
df_copy['time'] = df_copy['time'].astype(str)

# 2. Function to validate if time is in the correct "hhmm" format (e.g., "16h30")
def validate_time_format(time_str):
    # Regex pattern for valid time formats like "16h00", "01h50", etc.
    pattern = r'^\d{2}h\d{2}$'
    if re.match(pattern, time_str):
        return time_str
    else:
        return None  # Invalid format, will replace with mean later

# 3. Apply the validation function and keep only valid times
df_copy['time'] = df_copy['time'].apply(lambda x: x.replace('h', '') if validate_time_format(x) else None)

# 4. Convert the 'time_numeric' to numeric, replacing invalid parsing with None
df_copy['time'] = pd.to_numeric(df_copy['time'], errors='coerce')

# 5. Calculate the mean time, ignoring NaNs
mean_time = df_copy['time'].mean()

# 6. Fill NaN values (both invalid format and actual NaNs) with the calculated mean time
df_copy['time'].fillna(round(mean_time), inplace=True)

# 7. Convert all values in 'time_numeric' to integers
df_copy['time'] = df_copy['time'].astype(int)

# Output the updated dataframe
print(df)



                      date    year          type    country  \
0              15 Mar 2024  2024.0    Unprovoked  AUSTRALIA   
1              04 Mar 2024  2024.0    Unprovoked        USA   
2              02 Mar-2024  2024.0    Unprovoked        USA   
3              25 Feb-2024  2024.0    Unprovoked  AUSTRALIA   
4              14 Feb-2024  2024.0    Unprovoked      INDIA   
...                    ...     ...           ...        ...   
6737              Sep-1805  1805.0       Invalid        USA   
6738  Reported 26-Feb-1804  1804.0    Watercraft  AUSTRALIA   
6739           May-17-1803  1803.0  Sea Disaster        USA   
6740              Mar-1803  1803.0    Unprovoked  AUSTRALIA   
6741  Reported Apr-13-1802  1802.0    Unprovoked      INDIA   

                  state  activity  sex  age fatal   time  
0            Queensland  Swimming    F   13   NaN  16h00  
1                Hawaii   Surfing    M  NaN     N    NaN  
2                Hawaii  Swimming    F   11     N  13h30  
3     W

C:\Users\Marc Jay\AppData\Local\Temp\ipykernel_33100\3602328815.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['time'].fillna(round(mean_time), inplace=True)


In [9]:
# Cleaning 'age' column:

# Converting 'age' to numeric, setting errors to NaN:
df_copy['age'] = pd.to_numeric(df_copy['age'], errors='coerce')

# Calculating the mean age, excluding NaNs:
age_mean = df_copy['age'].mean()
print(f"Mean age: {age_mean}")

# Filling missing 'age' values with the calculated mean:
df_copy['age'] = df_copy['age'].fillna(age_mean)

# Rounding 'age' values to nearest whole number:
df_copy['age'] = df_copy['age'].round(0)

# Check:
df_copy['age'].value_counts(dropna=False)

Mean age: 27.718184429761564


age
28.0    2685
17.0     167
18.0     146
15.0     142
16.0     140
        ... 
72.0       1
84.0       1
86.0       1
87.0       1
81.0       1
Name: count, Length: 81, dtype: int64

In [10]:
def reformat_date_adjusted(date_str):
    # Ensure the input is a string; early return for empty or NaN-like strings
    date_str = str(date_str).strip()
    if not date_str or date_str.lower() == 'nan':
        return "invalid-date"
    # Normalize the date string
    date_str = re.sub(r'^Reported\s+', '', date_str, flags=re.IGNORECASE)
    date_str = re.sub(r'\s*-\s*|\s+', ' ', date_str)  # Convert '-' to ' ' and collapse multiple spaces
    # Try parsing the date with different formats
    try:
        # Detect format based on separators and content
        if re.search(r'\d{2}\s\w+\s\d{4}', date_str):  # DD MMM YYYY or DD MMMM YYYY
            for fmt in ("%d %b %Y", "%d %B %Y"):  # Try both abbreviated and full month name formats
                try:
                    parsed_date = datetime.strptime(date_str, fmt)
                    return parsed_date.strftime("%m-%Y")
                except ValueError:
                    pass  # If one format fails, try the next
        elif re.search(r'\w+\s\d{4}', date_str):  # MMM YYYY or MMMM YYYY
            for fmt in ("%b %Y", "%B %Y"):  # Try both abbreviated and full month name formats
                try:
                    parsed_date = datetime.strptime(date_str, fmt)
                    return parsed_date.strftime("%m-%Y")
                except ValueError:
                    pass
        elif re.match(r'\d{4}$', date_str):  # YYYY only
            return datetime.strptime(date_str, "%Y").strftime("%Y") + "-00"
    except Exception as e:
        print(f"Error processing date '{date_str}': {e}")
    return "Unknown"
# Application of the function:
df_copy['date'] = df_copy['date'].astype(str)
df_copy['date'] = df_copy['date'].apply(reformat_date_adjusted)
# Diagnostic check to review the transformation results
print(df_copy['date'].value_counts(dropna=False))

date
Unknown    193
06-2015     21
04-2017     21
09-2017     19
08-2014     19
          ... 
06-1827      1
1828-00      1
09-1828      1
1829-00      1
06-1829      1
Name: count, Length: 1684, dtype: int64


In [11]:
# Seasonality function and new seasonality column

def get_seasonality(formatted_date):
    try:
        month = int(formatted_date.split('-')[0])
        if month in [12, 1, 2]:
            return "Winter"
        elif month in [3, 4, 5]:
            return "Spring"
        elif month in [6, 7, 8]:
            return "Summer"
        elif month in [9, 10, 11]:
            return "Autumn"
        else:
            return "Unknown"
    except:
        return "Unknown"

df_copy['seasonality'] = df_copy['date'].apply(get_seasonality)

In [12]:
# Seasonality stats
valid_seasons_df = df_copy[df_copy['seasonality'] != "Unknown"]
season_counts = valid_seasons_df['seasonality'].value_counts()
total_count = len(valid_seasons_df)
season_percentage = (season_counts / total_count) * 100

print(season_percentage)

seasonality
Summer    30.602837
Autumn    24.645390
Winter    23.226950
Spring    21.524823
Name: count, dtype: float64


In [13]:
# Replace 'NaN' with 'unknown':
df_copy['activity'].fillna('unknown', inplace=True)

# Check:
df_copy['activity'].value_counts(dropna=False)


C:\Users\Marc Jay\AppData\Local\Temp\ipykernel_33100\1800121191.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['activity'].fillna('unknown', inplace=True)


activity
Surfing                    1101
Swimming                    932
unknown                     502
Spearfishing                322
Fishing                     268
                           ... 
Jumped into river             1
Wreck of the USS Somers       1
Wreck of the Tweed            1
Wreck of the Sovereign        1
Hilo                          1
Name: count, Length: 1358, dtype: int64

In [14]:
# Replace NaNs in the State column
df["state"].fillna("Unknown")

0              Queensland
1                  Hawaii
2                  Hawaii
3       Western Australia
4             Maharashtra
              ...        
6737             New York
6738      New South Wales
6739       South Carolina
6740    Western Australia
6741              Unknown
Name: state, Length: 6084, dtype: object

In [15]:
# Join 'country' and 'state' into 'location' to preserve data and enhance precision.
location = df_copy['country'].astype(str) + ', ' + df_copy['state'].astype(str)
df_copy = df_copy.join(location.rename('location'))
df_copy

,date,year,type,country,state,activity,sex,age,fatal,time,seasonality,location
0,03-2024,2024.0,Unprovoked,AUSTRALIA,Queensland,Swimming,f,13.0,n,1600,Spring,"AUSTRALIA, Queensland"
1,03-2024,2024.0,Unprovoked,USA,Hawaii,Surfing,m,28.0,n,1320,Spring,"USA, Hawaii"
2,03-2024,2024.0,Unprovoked,USA,Hawaii,Swimming,f,11.0,n,1330,Spring,"USA, Hawaii"
3,02-2024,2024.0,Unprovoked,AUSTRALIA,Western Australia,unknown,f,46.0,n,1130,Winter,"AUSTRALIA, Western Australia"
4,02-2024,2024.0,Unprovoked,INDIA,Maharashtra,Fishing,m,32.0,n,1320,Winter,"INDIA, Maharashtra"
...,...,...,...,...,...,...,...,...,...,...,...,...
6737,09-1805,1805.0,Invalid,USA,New York,unknown,m,28.0,n,1320,Autumn,"USA, New York"
6738,02-1804,1804.0,Watercraft,AUSTRALIA,New South Wales,unknown,NaN,28.0,n,1320,Winter,"AUSTRALIA, New South Wales"
6739,Unknown,1803.0,Sea Disaster,USA,South Carolina,unknown,m,28.0,n,1320,Unknown,"USA, South Carolina"
6740,03-1803,1803.0,Unprovoked,AUSTRALIA,Western Australia,unknown,m,28.0,n,1320,Spring,"AUSTRALIA, Western Australia"


In [16]:
# Subsequent overview of v0.5:
# Cell to be (re)moved or updated as we refine our analysis and finalize other KPIs.

print("head:")
print(df_copy.head())

print("\ninfo:")
df_copy.info()

print("\ndescribe:")
print(df_copy.describe())

print("\ncolumns:")
print(df_copy.columns.tolist())

print("\nmissing values:")
print(df_copy.isnull().sum()[df_copy.isnull().sum() > 0])

# Pending Cleaning Columns: 'date', 'year', 'activity', 'country', 'state'.
# Note on Geo Data: Considering merging 'country' and 'state' into 'location' to preserve data and enhance precision.

# Upcoming in v0.6:
# - Cleaning for 'date', 'year', 'type' and their integration.
# - Decision pending on creating a 'location' column and its integration.

# Uniform Commentary: Please keep documentation clear and accessible.

head:
      date    year        type    country              state  activity sex  \
0  03-2024  2024.0  Unprovoked  AUSTRALIA         Queensland  Swimming   f   
1  03-2024  2024.0  Unprovoked        USA             Hawaii   Surfing   m   
2  03-2024  2024.0  Unprovoked        USA             Hawaii  Swimming   f   
3  02-2024  2024.0  Unprovoked  AUSTRALIA  Western Australia   unknown   f   
4  02-2024  2024.0  Unprovoked      INDIA        Maharashtra   Fishing   m   

    age fatal  time seasonality                      location  
0  13.0     n  1600      Spring         AUSTRALIA, Queensland  
1  28.0     n  1320      Spring                   USA, Hawaii  
2  11.0     n  1330      Spring                   USA, Hawaii  
3  46.0     n  1130      Winter  AUSTRALIA, Western Australia  
4  32.0     n  1320      Winter            INDIA, Maharashtra  

info:
<class 'pandas.core.frame.DataFrame'>
Index: 6084 entries, 0 to 6741
Data columns (total 12 columns):
 #   Column       Non-Null Count